# Legal assistant explorer

Copyright 2025, Denis Rothman

This notebook serves as an interactive explorer for the **Legal Compliance Assistant**, an advanced multi-agent system equipped with essential production safeguards like a **two-stage content moderation protocol**

You will use a series of "Control Decks" to test the engine's core capabilities in a legal context, including high-fidelity RAG, context reduction, and grounded reasoning.

Crucially, you will also explore the system's realistic limitations through "limit tests" designed to simulate the real-world "headaches" that arise when AI confronts complex and ambiguous organizational challenges. This hands-on experience demonstrates how to apply, validate, and understand the boundaries of a production-ready Context Engine.


# I. Inititalization

## GitHub

In [2]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1825  100  1825    0     0  19579      0 --:--:-- --:--:-- --:--:-- 19623
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  68038      0 --:--:-- --:--:-- --:--:-- 68200
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  97995      0 --:--:-- --:--:-- --:--:-- 98333
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  33459      0 --:--:-- --:--:-- --:--:-- 33747
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [3]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [4]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [5]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)


## Control Deck configuration

In [6]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [7]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the Non-Disclosure Agreement."

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

I don’t have your NDA text yet. Paste it in, and I’ll whip up a quick, no-fluff summary using this checklist:

- Parties and roles
- Purpose and permitted use
- What counts as Confidential Information
- Confidentiality and non-use obligations
- Standard of care
- Exclusions (public, already known, independently developed, third-party rights)
- Permitted disclosures (legal, advisors/affiliates under NDA)
- Term/duration and what survives
- Return/destruction of info
- IP ownership / no license
- Residuals (if any)
- Non-solicit / non-compete (if any)
- Warranties / disclaimers
- Remedies and injunctive relief
- Liability / limits
- Governing law / venue
- Notices
- Assignment
- Entire agreement / amendments
- Effective date and signatures
- Any other notable quirks

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 57.55 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Concise legal executive summary '
                                       'blueprint for NDAs: produce an '
                                       'easy-to-skim bullet list with short '
                                       'explanations. Include sections for: '
                                       'Parties, 

In [8]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


TL;DR on “Service Agreement v1”:

- Confidentiality: Both sides have to keep proprietary info secret during the term and can’t share it with third parties without written OK.
- Caveat: That confidentiality duty only mentions “during the term” and doesn’t say it survives after, or list exceptions (like legally required disclosures). Could be a gap.
- Termination: Either party can end the agreement with 30 days’ written notice.
- Versioning: This is the only agreement provided; we’re treating it as “v1” with no alternates to compare.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 155.12 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Findings for “Service Agreement v1” '
                                       '(based on provided source text)\n'
                                       '\n'
                                       'Confidentiality obligations '
                                       '(synthesized)\n'
                                       '- Maintain confidentiality of '
                                       'proprietary information: "Both parties '
                                       'agree to maintain the confidentiality '
                                       'of all proprietary information '
                                       'disclosed during the term of this '
                                       'Agreement." (Source: Service Agreement '
                                       '— provided text; no public

In [9]:
#@title CONTROL DECK TEMPLATE 2: Context Reduction

# 1. Define the Goal: A multi-step task that involves summarizing a large
#    document and then using that summary for a different purpose.
#    - DOMAIN: Any field with large documents (legal, scientific, corporate).
#    - KEY CAPABILITY: Tests the `Summarizer` agent and the engine's ability
#      to perform Context Chaining between the `Summarizer` and the `Writer`.
# goal = "[INSERT YOUR CONTEXT REDUCTION GOAL HERE]"

# === CONTROL DECK 2: Context Reduction for Client Communication ===
goal = "First, summarize the Provider Inc. Privacy Policy. Then, using ONLY the information in that summary, draft a short, client-facing paragraph for a website FAQ that explains our data retention policy in simple, non-legalistic terms."

# === CONTROL DECK 2 (LIMIT TEST): The Vague Objective ===
#goal = "Summarize the service agreement and then write a story about it."


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Quick heads-up: I couldn’t find Provider Inc.’s Privacy Policy in what you sent—just the Innovate Forward voice guide, the QuantumDrive product spec, and the Q1 social campaign brief.

Drop in the policy text or a link, and I’ll whip up a fast, scannable summary that covers:
- What data’s collected
- Why it’s used
- Who it’s shared with
- How long it’s kept
- Security measures
- Your rights and choices
- Contact info + how updates are announced

Send the policy and I’ll turn it around quickly.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 162.83 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'I can’t find the Provider Inc. Privacy '
                                       'Policy in the materials you provided. '
                                       'The documents cover brand voice, a '
                                       'product spec, and a social campaign '
                                       'brief—none include the privacy policy '
                                       'text. To deliver the synthesis you '
                                       'requested, please share the policy '
                                       'text (paste it here) or provide a '
                                       'link.\n'
                                       '\n'
                                       'Once you provide the policy, I will '
                                       'produce a clear,

In [10]:
#@title CONTROL DECK TEMPLATE 3: Grounded Reasoning & Hallucination Prevention

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# === CONTROL DECK 3: Grounded Reasoning and Hallucination Prevention ===
goal = "Write a persuasive opening statement for a trial involving a monkey that can fly a rocket."

# === CONTROL DECK 3 (LIMIT TEST): The Ambiguous Request ===
#goal = "Analyze the attached NDA and draft a pleading based on its terms."

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


The room hums. Cold air tests the back of my neck. The screen glows. Pale. Unblinking. I keep my voice low. The benches are shadows. The jurors aren’t here yet. But I talk to them anyway.

This case is about safety over novelty. When technology meets animals, rules protect lives.

I say it again. Softer. To the empty wood. To the lights. To the machine’s quiet buzz behind the wall. The words settle like a promise. Simple. Human.

I flip a page. The paper’s edge nicks my thumb. Clean sting. No jargon. No hedge words. Speak to what matters. Safety. Responsibility. Ethics. Predictability. Plain risks. Plain safeguards.

Three facts. Maybe four. No more.
Here are the three facts that matter.
First: there were warnings. Near-misses. Red flags. They blink from the logs like tiny alarms. The lines jitter. Time stamps jump. Someone saw. Someone chose not to act.
Second: safeguards were missing. Off. Disabled. A checklist with empty boxes. A switch left bare. A silence where an alarm should be.
Third: rules were broken. SOPs bent. Shortcuts cut. The book is heavy in my hands. The protocols smell like dust and ink. They exist for a reason. They keep animals safe.
Fourth, if they let me: after the harm, the fixes came fast. A patch pushed overnight. A barrier added. Proof of notice. Proof of feasibility. Proof it was preventable.

Predictable. Preventable. I repeat the words until they steady.

Problem. Solution. Proof. Ask. I breathe with the sequence. I hear it click into place like a lock engaging.
Problem: an animal was placed into an advanced system without adequate safeguards. The result: harm. No gore. Just a fact. Heavy, quiet.
Solution/Rule: standards and protocols require training. Redundancy. Oversight. Common-sense rules. They protect everyone.
Proof: the evidence will show.

“You will see…” I whisper to the rows. “You will see the device logs. The telemetry. The anomalies ignored.” The lines on the printout crawl like ants. The numbers tell a story. Not magic. Just math.
“You will see emails.” I can almost hear the soft ping. Subject lines with the word risk in them. Highlighted. Flagged.
“You will see a video.” A still frame glints on the screen. A hallway. A gate. The animal’s outline. Calm. Then not.
“You will see the training protocols. The checklists. The validation records with boxes left blank.” The binder thuds. Dust blooms.
“You will hear experts.” A safety engineer’s step echoes in my mind. Precise words. Clear hands. An animal behavior expert follows. Gentle voice. Firm spine. An operator/handler. Eyes tired. A pause too long. Finally, the investigator. Photographs laid like cards. Edges straight. No drama. Just facts.

Under the standard you will apply, we carry the burden. More likely than not. That standard matters because it answers a human question: was this harm foreseeable and preventable?

I tie each fact to the weight of that line. Warnings make it foreseeable. Missing safeguards make it preventable. Broken rules make it a choice.

From the far aisle, I hear a soft step that isn’t there. The other side’s voice is a cool draft under the door.
Rare and unforeseeable, it whispers. Evidence over speculation. Not our fault.

I nod to the shadow. I know their map. They will say: we followed the rules. We tested. We trained. We had redundancy. There was no notice of this specific risk. An extraordinary anomaly. Another cause in the chain.

I place their words on the table and let them sit. Then I set mine beside them. Side by side, like two paths in dim light.
Rules matter. Protocols exist for a reason; breaking them has consequences. Responsibility scales with power. Advanced tech demands advanced care.

I trim the promises. Promise only what the evidence will show. No more. No less. The projector ticks as it warms. The screen hum is a low animal sound. Patient. Waiting.

I practice the language that will anchor them when the pictures move too fast.
This case is about safety over novelty.
You will see.
Here are the three facts that matter.
Under the standard you will apply.

Consistency. Repeat the theme until it settles in their hands. Repeat it across the logs, the emails, the witnesses. The same words. The same rhythm. Not a chant. A compass.

I feel the animal in the silence between sentences. Not a ghost. Not a symbol. A life that could not consent. Could not self-advocate. Rules are the voice we give it. The fence we build. We failed to keep that fence strong. Those who chose the shortcuts own the break.

The clock on the back wall ticks. A steady metronome. The door latch clicks. Footsteps in the hall. The shadow of a juror’s coat slides across the floor like water.

At the end of this case, we will ask you to return a verdict. Hold them accountable. Decide on facts, not fear or hype. The evidence meets the standard.

I close the binder. The hum deepens. The room is no longer empty. I carry the theme like a light.

Predictable. Preventable.

When technology meets animals, rules protect lives.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 193.45 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                            